In [1]:
from src.harness import architecture as arch
from src.harness import evolution as evo
from src.harness import utils

import copy
from enum import Enum
import functools
from importlib import reload
import itertools
from matplotlib import pyplot as plt
import multiprocess as mp
import numpy as np
import tensorflow as tf
from tensorflow import keras
from typing import Any, Callable, Dict, Iterable, List, Literal, Set, Tuple

reload(arch)
reload(evo)

2024-11-14 21:14:24.866071: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-14 21:14:24.943281: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-14 21:14:24.976359: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-14 21:14:25.134519: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-14 21:14:30.205444: W tensorflow/comp

<module 'src.harness.evolution' from '/gpfs1/home/j/b/jbourde2/lottery-tickets/src/harness/evolution.py'>

In [2]:
architecture = arch.Architecture("lenet", "mnist")

model_features = [
    evo.ModelFeatures.layer_sparsity, 
    evo.ModelFeatures.magnitude,
    evo.ModelFeatures.random,
    functools.partial(evo.ModelFeatures.synaptic_flow, loss_fn=keras.losses.CategoricalCrossentropy()),
]
arch_feature_selectors = [
    evo.ArchFeatures.layer_num,
    evo.ArchFeatures.layer_ohe,
    evo.ArchFeatures.layer_prop_params,
]

individual = evo.Individual("lenet", "mnist", model_features, arch_feature_selectors, [])

2024-11-14 21:14:40.684746: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31134 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:66:00.0, compute capability: 7.0


In [3]:
num_runs = 3

model_feature_selectors = [
    evo.ModelFeatures.layer_sparsity, 
    evo.ModelFeatures.magnitude,
    evo.ModelFeatures.random,
    functools.partial(evo.ModelFeatures.synaptic_flow, loss_fn=keras.losses.CategoricalCrossentropy()),
]
arch_feature_selectors = [
    evo.ArchFeatures.layer_num,
    evo.ArchFeatures.layer_ohe,
    evo.ArchFeatures.layer_prop_params,
]

layers = []

individual_constructor = functools.partial(
    evo.Individual, 
    architecture_name="lenet",
    dataset_name="mnist",
    model_feature_selectors=model_feature_selectors,
    arch_feature_selectors=arch_feature_selectors,
    layers=layers,
)

objectives = [
    (evo.Target.MAXIMIZE, lambda x: 1, evo.Individual.eval_accuracy),
    (evo.Target.MINIMIZE, lambda x: 1, evo.Individual.sparsity),
]

rate_func = lambda n: 0.5
scale_func = lambda n: 1 / np.sqrt(n + 1)
mutations = [
    functools.partial(evo.Individual.get_annealing_mutate(), rate=rate_func, scale=scale_func),
    evo.Individual.update_phenotype,
]
    
genome_metric_callbacks = [
]
kwargs = {
    "num_generations": 2,
    "archive_size": 10,
    "population_size": 10,
    "fronts_to_consider": 2,
    "tournament_size": 4,
    "num_tournament_winners": 2,
    "individual_constructor": individual_constructor,
    "objectives": objectives,
    "mutations": mutations,
    "crossover": evo.Individual.crossover,
    "genome_metric_callbacks": genome_metric_callbacks,
}

all_genome_metrics = []
all_objective_metrics = []
all_archives = []
for run in range(num_runs):
    print(f"Run {run + 1}")
    genome_metrics, objective_metrics, archive = evo.nsga2(**kwargs)
    all_genome_metrics.append(genome_metrics)
    all_objective_metrics.append(objective_metrics)
    all_archives.append(archive)

Run 1
Generation 1


/users/j/b/jbourde2/.conda/envs/lt/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
I0000 00:00:1731636890.334676  170581 service.cc:146] XLA service 0x2b6f10001010 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731636890.334832  170581 service.cc:154]   StreamExecutor device (0): Tesla V100-SXM2-32GB, Compute Capability 7.0
2024-11-14 21:14:50.372704: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-11-14 21:14:50.564389: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8906
I0000 00:00:1731636891.591281  170581 device_compiler.h:188] Compiled cluster using XLA!  This line is 

/users/j/b/jbourde2/.conda/envs/lt/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 4 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Generation 2
Run 2
Generation 1
Generation 2
Run 3
Generation 1
Generation 2


In [4]:
all_objective_metrics

[{'objective_0_value': array([[0.08, 0.08],
         [0.06, 0.08],
         [0.03, 0.07],
         [0.07, 0.08],
         [0.08, 0.11],
         [0.14, 0.08],
         [0.08, 0.08],
         [0.06, 0.08],
         [0.08, 0.07],
         [0.08, 0.08]]),
  'objective_1_value': array([[2.73808184e-04, 4.59097558e-03],
         [6.56689547e-02, 1.31277897e-04],
         [1.17778778e-01, 2.74183264e-03],
         [1.18176362e-01, 1.19425378e-02],
         [4.09962117e-02, 3.66227823e-02],
         [1.11304902e-01, 2.85435655e-02],
         [7.32155583e-03, 5.25111586e-05],
         [8.33127039e-02, 1.12523911e-05],
         [1.17782529e-01, 1.59371366e-02],
         [1.17790030e-01, 6.00127527e-05]]),
  'objective_0_range': array([1., 1.]),
  'objective_1_range': array([1., 1.])},
 {'objective_0_value': array([[0.08, 0.08],
         [0.08, 0.08],
         [0.08, 0.08],
         [0.08, 0.08],
         [0.08, 0.08],
         [0.08, 0.08],
         [0.08, 0.08],
         [0.08, 0.08],
        

In [8]:
"2".split(",")

['2']